## Simpler code with basics of semi-mechanistic approach
Now aiming to include both susceptible depletion and a variable reproduction number, we'll target:
$\\i_t = R_t\sum_{\tau<t} i_\tau g_{t-\tau}$

In [ ]:
from typing import Dict
from scipy.stats import gamma
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'

In [ ]:
def get_gamma_params_from_mean_sd(req_mean: float, req_sd: float) -> Dict[str, float]:
    var = req_sd ** 2.0
    scale = var / req_mean
    a = req_mean / scale
    return {'a': a, 'scale': scale}

In [ ]:
# Model parameters
seed = 1.0
gen_time_sd = 1.5
gen_time_mean = 5.0

In [ ]:
# Generation time
times = 40
gamma_params = get_gamma_params_from_mean_sd(gen_time_mean, gen_time_sd)
gen_time_densities = np.diff(gamma.cdf(range(times + 1), **gamma_params))

In [ ]:
# Variable reproduction number
process_vals = [2.0, 2.0, 2.0, 2.0]
process_times = np.linspace(0.0, times, len(process_vals))
process_model_vals = np.interp(range(times), process_times, process_vals)

In [ ]:
# The model loop
inc = np.zeros(times)
inc[0] = seed
pop = 100.0
suscept = pop - seed
for t in range(1, times):
    this_inc = (inc[:t] * gen_time_densities[t-1::-1] * process_model_vals[:t]).sum() * suscept / pop
    inc[t] = this_inc
    suscept = max(suscept - this_inc, 0.0)

In [ ]:
pd.Series(inc).plot(labels={'index': 'day', 'value': 'incidence'})